In [33]:
import pandas as pd
import numpy as np
import glob
import os

In [34]:
path = r'./gitignore/sales/'
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
raw_data = pd.concat(li, axis=0, ignore_index=True)

In [38]:
manual_export = raw_data[raw_data['Payment Method'].apply(lambda row: len(str(row).split(','))) > 1].copy()
sales_data = raw_data[raw_data['Payment Method'].apply(lambda row: len(str(row).split(','))) == 1].copy()

In [40]:
manual_export.to_csv('./gitignore/split_payments_manualentry.csv', index=False)

In [41]:
sales_data['Date/Time'] = pd.to_datetime(sales_data['Date/Time']).dt.date

In [44]:
data = []

def sales_to_manager_export(row):
    entry = []
    if row['Payment Method'] == 'Cash':
        entry.append(str(row['Date/Time']))
        entry.append(f"GOSALES {row['Number']} CASH")
        entry.append(row['Total'])
        entry.append('cash')
        entry.append(row['Total'])
        data.append(entry)
    elif row['Payment Method'] == 'Debit Card':
        entry.append(str(row['Date/Time']))
        entry.append(f"GOSALES {row['Number']} DEBIT")
        entry.append(row['Total'])
        entry.append('bank')
        entry.append(row['Total'])
        data.append(entry)
    elif row['Payment Method'] == 'Credit Card':
        entry.append(str(row['Date/Time']))
        entry.append(f"GOSALES {row['Number']} CREDIT")
        entry.append(row['Total'])
        entry.append('bank')
        entry.append(row['Total'])
        data.append(entry)
    elif row['Payment Method'] == 'Others':
        entry.append(str(row['Date/Time']))
        entry.append(f"GOSALES {row['Number']} XFER")
        entry.append(row['Total'])
        entry.append('bank')
        entry.append(row['Total'])
        data.append(entry)

sales_data.apply(sales_to_manager_export, axis=1)

cols = ['Date', 'Narration','Lines.1.Credit', 'Lines.2.Account', 'Lines.2.Debit']
export = pd.DataFrame(data, columns=cols)
export['Lines.1.Account'] = 'sales'
export.to_csv('./gitignore/export.csv', index=False)